In [41]:
!pip install --upgrade tensorflow

    100% |################################| 62.0MB 375kB/s ta 0:00:011    53% |#################               | 33.1MB 2.3MB/s eta 0:00:13    59% |##################              | 36.7MB 1.6MB/s eta 0:00:16    81% |#########################       | 50.4MB 1.2MB/s eta 0:00:10
    100% |################################| 1.2MB 1.2MB/s ta 0:00:011
    100% |################################| 3.1MB 1.4MB/s ta 0:00:011
  Found existing installation: protobuf 3.5.2
    Uninstalling protobuf-3.5.2:
      Successfully uninstalled protobuf-3.5.2
  Found existing installation: tensorboard 1.6.0
    Uninstalling tensorboard-1.6.0:
      Successfully uninstalled tensorboard-1.6.0
  Found existing installation: tensorflow 1.4.0
    Uninstalling tensorflow-1.4.0:
      Successfully uninstalled tensorflow-1.4.0


In [6]:
import keras
import os
import cv2
import scipy
import numpy as np
from tqdm import tqdm
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation, BatchNormalization
from keras.models import Model
from keras import optimizers

In [2]:
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K

def swish(x):
    return (K.sigmoid(x) * x)

get_custom_objects().update({'swish': Activation(swish)})

In [10]:
# What data should we load?
# What to do with unlabeled data?
def get_data(folder):
    X, y, z = [], [], []
    wbc_types = {'NEUTROPHIL': (0,1), 'EOSINOPHIL': (1,1), 'MONOCYTE': (2,0), 'LYMPHOCYTE': (3,0)}
    for (wbc_type, labels) in wbc_types.items():  
        for image_filename in tqdm(os.listdir(folder + wbc_type)):
            img_file = cv2.imread(folder + wbc_type + '/' + image_filename)
            if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(60, 80, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(labels[0])
                    z.append(labels[1])
    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(z)
    return X,y,z

from keras.utils.np_utils import to_categorical

X_train, y_train, z_train = get_data("blood-cells/dataset2-master/images/TRAIN/")
X_test, y_test, z_test = get_data("blood-cells/dataset2-master/images/TEST/")
y_trainHot = to_categorical(y_train, num_classes = 4)
y_testHot = to_categorical(y_test, num_classes = 4)
z_trainHot = to_categorical(z_train, num_classes = 2)
z_testHot = to_categorical(z_test, num_classes = 2)

  0%|          | 0/2499 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/misc/pilutil.py:480: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/misc/pilutil.py:483: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):
100%|██████████| 620/620 [00:01<00:00, 412.94it/s]


In [ ]:
# data visualization etc.

In [19]:
def build_model(*args, **kwargs):
    inp = Input(shape=(60,80,3))
    x = BatchNormalization()(inp)
    x = Conv2D(4, 5, strides=(2, 2), padding='same', activation='swish')(x)
    x = MaxPool2D(pool_size=(2, 2), padding="same", strides=(2,2))(x)
    x = Conv2D(4, 5, strides=(2, 2), padding='same', activation='swish')(x)
    x = MaxPool2D(pool_size=(2, 2), padding="same", strides=(2,2))(x)
    x = Flatten()(x)
    x = Dense(16, activation='swish')(x)
    x = Dropout(0.2)(x)
    y = Dense(4, activation='softmax')(x)
    
    model = Model(inp, y)
    opt = optimizers.Adam(lr=0.01, decay=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer="adam",
                  metrics=['accuracy'])
    return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 60, 80, 3)         0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 60, 80, 3)         12        
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 40, 4)         304       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 20, 4)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 10, 4)          404       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 5, 4)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 80)                0         
__________

In [20]:
# train model
h = model.fit(X_train, y_trainHot, epochs=80, batch_size=32, validation_data=(X_test, y_testHot), verbose=1)

Train on 9957 samples, validate on 2487 samples
Epoch 1/80
9957/9957 [==============================] - 19s 2ms/step - loss: 1.3492 - acc: 0.3223 - val_loss: 1.2117 - val_acc: 0.4033
Epoch 2/80
9957/9957 [==============================] - 18s 2ms/step - loss: 1.1631 - acc: 0.4446 - val_loss: 1.0248 - val_acc: 0.5275
Epoch 3/80
9957/9957 [==============================] - 18s 2ms/step - loss: 0.9952 - acc: 0.5549 - val_loss: 0.7281 - val_acc: 0.6940
Epoch 4/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.7813 - acc: 0.6575 - val_loss: 0.6868 - val_acc: 0.7153
Epoch 5/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.7158 - acc: 0.6796 - val_loss: 0.6394 - val_acc: 0.7205
Epoch 6/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.6771 - acc: 0.6971 - val_loss: 0.6632 - val_acc: 0.7181
Epoch 7/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.6447 - acc: 0.7129 - val_loss: 0.6272 - val_acc: 0.7314
Epoch 8

Epoch 61/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.3115 - acc: 0.8614 - val_loss: 0.5598 - val_acc: 0.7961
Epoch 62/80
9957/9957 [==============================] - 21s 2ms/step - loss: 0.3104 - acc: 0.8633 - val_loss: 0.4681 - val_acc: 0.8098
Epoch 63/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.3079 - acc: 0.8624 - val_loss: 0.5212 - val_acc: 0.7965
Epoch 64/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.3192 - acc: 0.8579 - val_loss: 0.5026 - val_acc: 0.7925
Epoch 65/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.3052 - acc: 0.8647 - val_loss: 0.4629 - val_acc: 0.8094
Epoch 66/80
9957/9957 [==============================] - 20s 2ms/step - loss: 0.3101 - acc: 0.8640 - val_loss: 0.4625 - val_acc: 0.8162
Epoch 67/80
9957/9957 [==============================] - 19s 2ms/step - loss: 0.3155 - acc: 0.8610 - val_loss: 0.4559 - val_acc: 0.8243
Epoch 68/80
9957/9957 [=========================

In [18]:
# test model
model.evaluate(X_test, y_testHot, batch_size=128)

2487/2487 [==============================] - 2s 657us/step


[1.459717612023041, 0.5199034975674625]